In [15]:
import torch
import torch.nn as nn
import functools
import operator

class Conv2dSamePadding(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super(Conv2dSamePadding, self).__init__(*args, **kwargs)
        self.zero_pad_2d = nn.ZeroPad2d(functools.reduce(operator.__add__,
                  [(k // 2, k // 2) for k in self.kernel_size[::-1]]))

    def forward(self, input):
        return self._conv_forward(self.zero_pad_2d(input), self.weight, self.bias)

class DepthwiseConv2d(nn.Module):
    def __init__(self, in_channels, depth_multiplier, kernel_size=3, stride=1, padding='same', activation='relu'):
        super(DepthwiseConv2d, self).__init__()

        if padding == 'same':
            self.depthwise_conv = Conv2dSamePadding(
                in_channels=in_channels,
                out_channels=in_channels * depth_multiplier,
                kernel_size=kernel_size,
                stride=stride,
                groups=in_channels
            )
        else:
            self.depthwise_conv = nn.Conv2d(
                in_channels=in_channels,
                out_channels=in_channels * depth_multiplier,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                groups=in_channels
            )

        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'relu6':
            self.activation = nn.ReLU6()
        else:
            self.activation = None

    def forward(self, x):
        x = self.depthwise_conv(x)
        if self.activation is not None:
            x = self.activation(x)
        return x

# Example usage:
in_channels = 32
depth_multiplier = 1
kernel_size = 3
stride = 1
padding = 'same'
activation = 'relu'

# Create an instance of the DepthwiseConv2d layer
depthwise_conv_layer = DepthwiseConv2d(in_channels, depth_multiplier, kernel_size, stride, padding, activation)

# Example input: batch size of 1, 32 input channels, and 112x112 spatial dimensions
input_tensor = torch.randn(1, in_channels, 112, 112)

# Apply depthwise convolution
output_tensor = depthwise_conv_layer(input_tensor)

print(f"Input shape: {input_tensor.shape}")
print(f"Output shape: {output_tensor.shape}")  # Should be (1, 96, 112, 112)


Input shape: torch.Size([1, 32, 112, 112])
Output shape: torch.Size([1, 32, 112, 112])


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import functools
import operator

class Stem(nn.Module):
    def __init__(self):
        super(Stem, self).__init__()
        self.conv1 = Conv2dSamePadding(in_channels=3, out_channels=32, kernel_size=3, stride=2)  # Changed stride to 2
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = Conv2dSamePadding(in_channels=32, out_channels=32, kernel_size=3, stride=1)
        self.bn2 = nn.BatchNorm2d(32)

    def forward(self, x):
        print("Shape of Input of stem:", x.shape)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        print("Shape of Output of stem:", x.shape)
        return x

# Example usage
input_tensor = torch.randn(1, 3, 224, 224)  # Batch size 1, 3 channels, 224x224 image
model = Stem()
output = model(input_tensor)


Shape of Input of stem: torch.Size([1, 3, 224, 224])
Shape of Output of stem: torch.Size([1, 32, 112, 112])


In [37]:
class LKA(nn.Module):
    def __init__(self, in_channels, fltr, depth_multiplier=1):
        super(LKA, self).__init__()
        self.depthwise1 = DepthwiseConv2d(in_channels, depth_multiplier=depth_multiplier, kernel_size=3, stride=1, padding='same', activation='relu')
        self.bn1 = nn.BatchNorm2d(in_channels * depth_multiplier)
        self.depthwise2 = DepthwiseConv2d(in_channels * depth_multiplier, depth_multiplier=depth_multiplier, kernel_size=3, stride=1, padding='same', activation='relu')
        self.bn2 = nn.BatchNorm2d(in_channels * depth_multiplier)
        self.conv = Conv2dSamePadding(in_channels * depth_multiplier, fltr, kernel_size=1, stride=1)  # Output channels set to in_channels

    def forward(self, x):
        print("Shape of Input of lka:", x.shape)
        x1 = F.relu(self.bn1(self.depthwise1(x)))
        print("Shape of x1", x1.shape)
        x2 = F.relu(self.bn2(self.depthwise2(x1)))
        print("Shape of x2", x2.shape)
        x3 = F.relu(self.conv(x2))
        print("Shape of x3", x3.shape)
        out = x * x3
        print("Shape of Output of lka:", out.shape)
        return out

# Example usage
input_tensor = torch.randn(1, 32, 224, 224)  # Batch size 1, 3 channels, 224x224 image
model = LKA(in_channels=32, fltr=32, depth_multiplier=1)
output = model(input_tensor)
print(f"Input shape: {input_tensor.shape}")
print(f"Output shape: {output.shape}")


Shape of Input of lka: torch.Size([1, 32, 224, 224])
Shape of x1 torch.Size([1, 32, 224, 224])
Shape of x2 torch.Size([1, 32, 224, 224])
Shape of x3 torch.Size([1, 32, 224, 224])
Shape of Output of lka: torch.Size([1, 32, 224, 224])
Input shape: torch.Size([1, 32, 224, 224])
Output shape: torch.Size([1, 32, 224, 224])


In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GAB(nn.Module):
    def __init__(self, in_channels):
        super(GAB, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=1)

    def forward(self, inputs):
        print("Shape of Input of GAB:", inputs.shape)

        # Channel attention mechanism
        x = self.avg_pool(inputs)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = torch.sigmoid(x)

        C_A = x * inputs

        # Spatial attention mechanism
        x = torch.mean(C_A, dim=1, keepdim=True)
        x = torch.sigmoid(x)

        S_A = x * C_A
        print("Shape of Output of GAB:", S_A.shape)

        return S_A

# Example usage:
inputs = torch.randn(1, 32, 112, 112)  # Example input tensor with shape (batch_size, channels, height, width)
model = GAB(in_channels=32)
output = model(inputs)
print(output.shape)

Shape of Input of GAB: torch.Size([1, 32, 112, 112])
Shape of Output of GAB: torch.Size([1, 32, 112, 112])
torch.Size([1, 32, 112, 112])


In [19]:
class MSConv(nn.Module):
    def __init__(self, in_channels, out_channels, strides, dm):
        super(MSConv, self).__init__()

        self.depth_conv1 = DepthwiseConv2d(in_channels, depth_multiplier, kernel_size, stride, padding, activation)
        self.bn1 = nn.BatchNorm2d(in_channels)

        self.depth_conv2 = DepthwiseConv2d(in_channels, depth_multiplier, kernel_size, stride, padding, activation)
        self.bn2 = nn.BatchNorm2d(in_channels)

        self.depth_conv4 = DepthwiseConv2d(in_channels, depth_multiplier, kernel_size, stride, padding, activation)
        self.bn4 = nn.BatchNorm2d(in_channels)

        self.depth_conv3 = DepthwiseConv2d(in_channels, depth_multiplier, kernel_size, stride, padding, activation)
        self.bn3 = nn.BatchNorm2d(in_channels)

        self.depth_conv5 = DepthwiseConv2d(in_channels, depth_multiplier, kernel_size, stride, padding, activation)
        self.bn5 = nn.BatchNorm2d(in_channels)

        self.depth_conv6 = DepthwiseConv2d(in_channels, depth_multiplier, kernel_size, stride, padding, activation)
        self.bn6 = nn.BatchNorm2d(in_channels)

        self.conv = nn.Conv2d(in_channels * 3, out_channels, kernel_size=1)
        self.bn7 = nn.BatchNorm2d(out_channels)

    def forward(self, input):
        print("Shape of input of MSConv:", input.shape)

        x = self.depth_conv1(input)
        x = self.bn1(x)
        shape = x.shape

        y = self.depth_conv2(input)
        y = self.bn2(y)
        y = self.depth_conv4(y)
        y = self.bn4(y)

        z = self.depth_conv3(input)
        z = self.bn3(z)
        z = self.depth_conv5(z)
        z = self.bn5(z)
        z = self.depth_conv6(z)
        z = self.bn6(z)

        yr = F.interpolate(y, size=(shape[2], shape[3]), mode='bilinear', align_corners=True)
        zr = F.interpolate(z, size=(shape[2], shape[3]), mode='bilinear', align_corners=True)

        ot = torch.cat([x, yr, zr], dim=1)
        ot = self.conv(ot)
        ot = F.relu(ot)
        ot = self.bn7(ot)

        inr = F.interpolate(input, size=(ot.shape[2], ot.shape[3]), mode='bilinear', align_corners=True)
        out = ot + inr

        print("Shape of Output of MSConv:", out.shape)
        return out

# Example usage:
inputs = torch.randn(1, 32, 112, 112)  # Example input tensor with shape (batch_size, channels, height, width)
model = MSConv(in_channels=32, out_channels=32, strides=1, dm=1)
output = model(inputs)
print(output.shape)


Shape of input of MSConv: torch.Size([1, 32, 112, 112])
Shape of Output of MSConv: torch.Size([1, 32, 112, 112])
torch.Size([1, 32, 112, 112])


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DefConv(nn.Module):
    def __init__(self, fltr, strides, dm=1):
        super(DefConv, self).__init__()
        self.bn1 = nn.BatchNorm2d(fltr)
        self.conv1 = nn.Conv2d(fltr, fltr, kernel_size=1, padding=0)
        self.msconv = MSConv(fltr, fltr, strides, dm)
        self.gab = GAB(fltr)
        self.conv2 = nn.Conv2d(fltr, fltr, kernel_size=1, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.strides = strides

    def forward(self, input):
        print("Shape of input of defconv:", input.shape)
        x = self.bn1(input)
        x = self.conv1(x)
        x = F.gelu(x)
        x = self.bn1(x)
        x = self.msconv(x)
        x = self.gab(x)
        x = self.conv2(x)
        x = F.gelu(x)

        if self.strides == 2:
            input = self.pool(input)
            input = self.conv2(input)
            input = F.gelu(input)
            x = x + input
        else:
            input = self.conv2(input)
            input = F.gelu(input)
            x = x + input

        print("Shape of output of defconv:", x.shape)
        return x

# Example usage:
input_tensor = torch.randn(1, 32, 112, 112)
model = DefConv(fltr=32, strides=1, dm=1)
output = model(input_tensor)
print(output.shape)

Shape of input of defconv: torch.Size([1, 32, 112, 112])
Shape of input of MSConv: torch.Size([1, 32, 112, 112])
Shape of Output of MSConv: torch.Size([1, 32, 112, 112])
Shape of Input of GAB: torch.Size([1, 32, 112, 112])
Shape of Output of GAB: torch.Size([1, 32, 112, 112])
Shape of output of defconv: torch.Size([1, 32, 112, 112])
torch.Size([1, 32, 112, 112])


In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransEnc(nn.Module):
    def __init__(self, fltr, nl, nh):
        super(TransEnc, self).__init__()
        self.nl = nl
        self.norm1 = nn.LayerNorm(fltr)
        self.norm2 = nn.LayerNorm(fltr)
        self.conv = nn.Conv2d(fltr, fltr, kernel_size=1, padding=0)
        self.lka_attn = LKA(fltr, fltr, depth_multiplier=nh)  # Assuming `LKA` is your custom large kernel attention

    def forward(self, input):
        print("Shape of input of trans_enc:", input.shape)
        x = input
        for i in range(self.nl):
            x = x.permute(0, 2, 3, 1)  # Change shape from (N, C, H, W) to (N, H, W, C) for LayerNorm
            x = self.norm1(x)
            x = x.permute(0, 3, 1, 2)  # Change back to (N, C, H, W)
            x = self.lka_attn(x)
            x = x + input

            y = x.permute(0, 2, 3, 1)
            y = self.norm2(y)
            y = y.permute(0, 3, 1, 2)
            y = self.conv(y)
            y = F.relu(y)
            x = x + y

        print("Shape of output of trans_enc:", x.shape)
        return x

# Example usage:
input_tensor = torch.randn(1, 32, 112, 112)
model = TransEnc(fltr=32, nl=3, nh=1)  # Adjust the number of layers (nl) and heads (nh) as required
output = model(input_tensor)
print(output.shape)


Shape of input of trans_enc: torch.Size([1, 32, 112, 112])
Shape of Input of lka: torch.Size([1, 32, 112, 112])
Shape of x1 torch.Size([1, 32, 112, 112])
Shape of x2 torch.Size([1, 32, 112, 112])
Shape of x3 torch.Size([1, 32, 112, 112])
Shape of Output of lka: torch.Size([1, 32, 112, 112])
Shape of Input of lka: torch.Size([1, 32, 112, 112])
Shape of x1 torch.Size([1, 32, 112, 112])
Shape of x2 torch.Size([1, 32, 112, 112])
Shape of x3 torch.Size([1, 32, 112, 112])
Shape of Output of lka: torch.Size([1, 32, 112, 112])
Shape of Input of lka: torch.Size([1, 32, 112, 112])
Shape of x1 torch.Size([1, 32, 112, 112])
Shape of x2 torch.Size([1, 32, 112, 112])
Shape of x3 torch.Size([1, 32, 112, 112])
Shape of Output of lka: torch.Size([1, 32, 112, 112])
Shape of output of trans_enc: torch.Size([1, 32, 112, 112])
torch.Size([1, 32, 112, 112])


In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LKA_Attn(nn.Module):
    def __init__(self, fltr, nh):
        super(LKA_Attn, self).__init__()
        self.nh = nh
        self.convs = nn.ModuleList([
            nn.Conv2d(fltr, fltr, kernel_size=1, padding=0) for _ in range(nh)
        ])
        self.activations = [F.relu, F.gelu, torch.sigmoid]
        self.lka_layers = nn.ModuleList([LKA(fltr, fltr) for _ in range(nh)])
        self.final_convs = nn.ModuleList([
            nn.Conv2d(fltr, fltr, kernel_size=1, padding=0) for _ in range(nh)
        ])

    def forward(self, input):
        print("Shape of input of lka_attn:", input.shape)
        attn = []
        y = input
        for i in range(self.nh):
            x = self.convs[i](input)
            x = self.activations[i](x)
            x = self.lka_layers[i](x)
            x = self.final_convs[i](x)
            x = self.activations[i](x)
            x = x + y
            attn.append(x)

        mh_lka_attn = attn[0]
        for i in range(1, len(attn)):
            mh_lka_attn = mh_lka_attn + attn[i]

        print("Shape of output of lka_attn:", mh_lka_attn.shape)
        return mh_lka_attn

# Example usage:
input_tensor = torch.randn(1, 32, 112, 112)
model = LKA_Attn(fltr=32, nh=3)  # Adjust the number of heads (nh) as required
output = model(input_tensor)
print(output.shape)


Shape of input of lka_attn: torch.Size([1, 32, 112, 112])
Shape of Input of lka: torch.Size([1, 32, 112, 112])
Shape of Output of lka: torch.Size([1, 32, 112, 112])
Shape of Input of lka: torch.Size([1, 32, 112, 112])
Shape of Output of lka: torch.Size([1, 32, 112, 112])
Shape of Input of lka: torch.Size([1, 32, 112, 112])
Shape of Output of lka: torch.Size([1, 32, 112, 112])
Shape of output of lka_attn: torch.Size([1, 32, 112, 112])
torch.Size([1, 32, 112, 112])


In [23]:
import torch
import torch.nn as nn

class DefVitBlock(nn.Module):
    def __init__(self, fltr, strides, nl, nh, dm=1):
        super(DefVitBlock, self).__init__()
        self.defconv = DefConv(fltr, strides, dm)
        self.trans_enc = TransEnc(fltr, nl, nh)

    def forward(self, input):
        print("Shape of input of DefVitBlock:", input.shape)
        x = self.defconv(input)
        x = self.trans_enc(x)
        print("Shape of output of DefVitBlock:", x.shape)
        return x

# Example usage:
input_tensor = torch.randn(1, 32, 112, 112)
model = DefVitBlock(fltr=32, strides=1, nl=1, nh=1)  # Adjust the parameters as required
output = model(input_tensor)
print(output.shape)

Shape of input of DefVitBlock: torch.Size([1, 32, 112, 112])
Shape of input of defconv: torch.Size([1, 32, 112, 112])
Shape of input of MSConv: torch.Size([1, 32, 112, 112])
Shape of Output of MSConv: torch.Size([1, 32, 112, 112])
Shape of Input of GAB: torch.Size([1, 32, 112, 112])
Shape of Output of GAB: torch.Size([1, 32, 112, 112])
Shape of output of defconv: torch.Size([1, 32, 112, 112])
Shape of input of trans_enc: torch.Size([1, 32, 112, 112])
Shape of Input of lka: torch.Size([1, 32, 112, 112])
Shape of Output of lka: torch.Size([1, 32, 112, 112])
Shape of output of trans_enc: torch.Size([1, 32, 112, 112])
Shape of output of DefVitBlock: torch.Size([1, 32, 112, 112])
torch.Size([1, 32, 112, 112])


In [29]:
# DefVit model
class DefVit(nn.Module):
    def __init__(self, input_shape, num_classes):
        super(DefVit, self).__init__()
        self.stem = Stem()
        self.stage1_block1 = DefVitBlock(32, strides=1, nl=2, nh=3)
        self.stage1_block2 = DefVitBlock(32, strides=1, nl=2, nh=3)
        self.stage2_block1 = DefVitBlock(64, strides=1, nl=2, nh=3)
        self.stage2_block2 = DefVitBlock(64, strides=1, nl=2, nh=3)
        self.stage3_block1 = DefVitBlock(128, strides=1, nl=4, nh=3)
        self.stage3_block2 = DefVitBlock(128, strides=1, nl=4, nh=3)
        self.stage3_block3 = DefVitBlock(128, strides=1, nl=4, nh=3)
        self.stage3_block4 = DefVitBlock(128, strides=1, nl=4, nh=3)
        self.stage3_block5 = DefVitBlock(128, strides=1, nl=4, nh=3)
        self.stage4_block1 = DefVitBlock(256, strides=1, nl=2, nh=3)
        self.stage4_block2 = DefVitBlock(256, strides=1, nl=2, nh=3)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.stem(x)
        print('After stage-0', x.shape)

        x = self.stage1_block1(x)
        x = self.stage1_block2(x)
        x = F.max_pool2d(x, kernel_size=2)
        print('After stage-1', x.shape)

        x = self.stage2_block1(x)
        x = self.stage2_block2(x)
        x = F.max_pool2d(x, kernel_size=2)
        print('After stage-2', x.shape)

        x = self.stage3_block1(x)
        x = self.stage3_block2(x)
        x = self.stage3_block3(x)
        x = self.stage3_block4(x)
        x = self.stage3_block5(x)
        x = F.max_pool2d(x, kernel_size=2)
        print('After stage-3', x.shape)

        x = self.stage4_block1(x)
        x = self.stage4_block2(x)
        x = F.max_pool2d(x, kernel_size=2)
        print('After stage-4', x.shape)

        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Example usage:
input_tensor = torch.randn(1, 3, 224, 224)
num_classes = 8
model = DefVit(input_shape=(3, 224, 224), num_classes=num_classes)
output = model(input_tensor)
print(output.shape)

Shape of Input of stem: torch.Size([1, 3, 224, 224])
Shape of Output of stem: torch.Size([1, 32, 112, 112])
After stage-0 torch.Size([1, 32, 112, 112])
Shape of input of DefVitBlock: torch.Size([1, 32, 112, 112])
Shape of input of defconv: torch.Size([1, 32, 112, 112])
Shape of input of MSConv: torch.Size([1, 32, 112, 112])
Shape of Output of MSConv: torch.Size([1, 32, 112, 112])
Shape of Input of GAB: torch.Size([1, 32, 112, 112])
Shape of Output of GAB: torch.Size([1, 32, 112, 112])
Shape of output of defconv: torch.Size([1, 32, 112, 112])
Shape of input of trans_enc: torch.Size([1, 32, 112, 112])
Shape of Input of lka: torch.Size([1, 32, 112, 112])
Shape of x1 torch.Size([1, 96, 112, 112])


RuntimeError: running_mean should contain 288 elements not 96